# Segmenting and Clustering Neighborhoods in Toronto | Part-1  


1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

3. To create the above dataframe:

. 
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
.


4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

Importing Libraries and data import from wikipedia

In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extr_data = requests.get(url).text
data = BeautifulSoup(extr_data, 'lxml')
data

:_L" title="List of postal codes of Canada: L">L</a>
</td>
<td align="center" style="border: 1px solid #E0FFF8; background-color: #00FFC0; font-size: 135%; color: black;" width="5%"><a class="mw-selflink selflink">M</a>
</td>
<td align="center" style="border: 1px solid #00FFE0; background-color: #E0FFFC; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_N" title="List of postal codes of Canada: N">N</a>
</td>
<td align="center" style="border: 1px solid #00FFFF; background-color: #E0FFFF; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_P" title="List of postal codes of Canada: P">P</a>
</td>
<td align="center" style="border: 1px solid #00C0FF; background-color: #E0F8FF; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_R" title="List of postal codes of Canada: R">R</a>
</td>
<td align="center" style="border: 1px solid #0080FF; background-color: #E0F0FF; font-size: 135%;" width="5%"><a href="/wiki/List_of_pos

Converting data as dataframe

In [43]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    k = 0
    for td in tr.find_all('td'):
        if k == 0:
            postcode = td.text
            k = k + 1
        elif k == 1:
            borough = td.text
            k = k + 1
        elif k == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
toronto

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A\n,Not assigned\n,Not assigned
2,M2A\n,Not assigned\n,Not assigned
3,M3A\n,North York\n,Parkwoods
4,M4A\n,North York\n,Victoria Village
...,...,...,...
176,M5Z\n,Not assigned\n,Not assigned
177,M6Z\n,Not assigned\n,Not assigned
178,M7Z\n,Not assigned\n,Not assigned
179,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


Data cleaning

In [37]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
k = 0
for k in range(0,toronto.shape[0]):
    if toronto.iloc[k][2] == 'Not assigned':
        toronto.iloc[k][2] = toronto.iloc[k][1]
        k = k+1
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
...,...,...,...
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"
177,M9X\n,Not assigned\n,Not assigned\n
178,M9Y\n,Not assigned\n,Not assigned\n


In [23]:
print(df.describe())

       Postalcode         Borough    Neighborhood
count         180             180             180
unique        180              11             100
top         M7M\n  Not assigned\n  Not assigned\n
freq            1              77              77


Drop None rows and rows which contains 'Not assigned' values ("Not assigned" are replaced by 'NaN')

In [24]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [25]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [26]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [27]:
print(df2.describe())

       Postalcode         Borough    Neighborhood
count         180             180             180
unique        180              11             100
top         M7M\n  Not assigned\n  Not assigned\n
freq            1              77              77


In [28]:
print(df2.shape)
print(df2.head())

(180, 3)
  Postalcode         Borough                            Neighborhood
0      M1A\n  Not assigned\n                          Not assigned\n
1      M1B\n   Scarborough\n                          Malvern, Rouge
2      M1C\n   Scarborough\n  Rouge Hill, Port Union, Highland Creek
3      M1E\n   Scarborough\n       Guildwood, Morningside, West Hill
4      M1G\n   Scarborough\n                                  Woburn


In [29]:
df2.to_csv('toronto part1.csv', index=False) 